In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sklearn
print(sklearn.__version__)
# Debe ser version 1.2.2 de lo contrario ejecutar:
#!pip install scikit-learn==1.2.2

1.2.2


In [3]:
def metrics(y_test, predicciones):

    # Coeficiente de determinación (R-cuadrado)
    r2 = r2_score(y_test, predicciones)
    # Error cuadrado medio
    rmse = mean_squared_error(y_true  = y_test, y_pred  = predicciones, squared = False)
    # Error Absoluto Medio
    mae = mean_absolute_error(y_test, predicciones)
    # Error Porcentual Absoluto Medio
    # mape = np.mean(np.abs((y_test - predicciones) / y_test)) * 100
    epsilon=1e-10
    mape = np.mean(np.abs((y_test - predicciones) / (y_test + epsilon))) * 100

    print("R^2:", r2)
    print("RMSE", rmse)
    print("MAE:", mae)
    print("MAPE:", mape, "%")

    r2 = round(r2, 6)
    rmse = round(rmse, 6)
    mae = round(mae, 6)
    mape = round(mape, 6)
    return r2, rmse, mae, mape

# Todos los modelos

In [4]:
import openpyxl
import numpy as np
import time
import os
import pandas as pd
import joblib
import json
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from math import sqrt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from joblib import dump
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error, r2_score

# RNN
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.base import BaseEstimator, RegressorMixin
from tensorflow.keras.callbacks import ModelCheckpoint

import pickle
import pytz

In [5]:
# Obtener la hora actual en UTC
utc_now = datetime.now(pytz.utc)
mexico_city_tz = pytz.timezone('America/Mexico_City')
mexico_city_now = utc_now.astimezone(mexico_city_tz)
hora_formateada = mexico_city_now.strftime('%Y-%m-%d %H:%M')
print("Hora actual en Ciudad de México:", hora_formateada)


Hora actual en Ciudad de México: 2024-06-07 12:58


In [6]:
def leer_archivo(station, target):

  #file = files[3]
  file_path = os.path.join(dir,station)
  df  = pd.read_csv(file_path)
  dates = df.date
  y = df[target]
  X = df.drop(columns=['date', 'year', 'month', 'day', 'hour', 'minute'])
  X = X.drop(columns=[target])

  return X, y, df, file_path


# Convertir los datos en secuencias adecuadas para RNN
def create_sequences(X, y, time_steps, time_future):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps+ time_future])
    return np.array(Xs), np.array(ys)


def create_sequences2(X, y, time_steps, time_future):
  Xs, ys = [], []
  for i in range(len(X) - time_steps-time_future):
    df = X[i:(i + time_steps)]
    array = df.to_numpy()
    # Aplanar el array a un vector
    vector = array.flatten()
    Xs.append(vector)
    ys.append(y[i + time_steps+time_future])
  return np.array(Xs), np.array(ys)

def MLP_grid(X,y, station_name, fecha, best_model_dir, best_param_dir, time_steps, time_future, results_dir):

  X_seq, y_seq = create_sequences2(X, y, time_steps, time_future)
  # Dividir los datos en conjunto de entrenamiento y prueba
  X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
  # Dividir los datos en conjunto de entrenamiento y prueba
  #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Definir el regresor MLP
  #mlp = MLPRegressor(max_iter=1000)
  mlp = MLPRegressor(max_iter=1000)

  # Definir el diccionario de parámetros para GridSearchCV
  param_grid = {
      'hidden_layer_sizes': [(10,), (50,), (100,), (50, 50), (100,100), (50,50,50), (100,100,100), (75,75,75,75), (25,25,25,25), (100,150,100,25)],
      'activation': ['relu', 'tanh'],
      'solver': ['adam', 'sgd'],
      'alpha': [0.0001, 0.05],
      'learning_rate': ['constant', 'adaptive'],
      'max_iter': [200, 500, 1000, 1500],  # Especificar diferentes números de iteraciones
  }

  # Configurar GridSearchCV

  #'neg_mean_squared_error'
  grid_search = GridSearchCV(mlp, param_grid, scoring='r2', cv=5, n_jobs=-1, verbose=1 ,return_train_score=True)
  # Ajustar GridSearchCV a los datos de entrenamiento
  grid_search.fit(X_train, y_train)

  # Mejores parámetros encontrados
  print("Mejores parámetros:", grid_search.best_params_)

  # Obtener el mejor modelo entrenado
  best_model = grid_search.best_estimator_

  # Predecir en el conjunto de prueba
  y_pred = best_model.predict(X_test)

  # Calcular el RMSE en el conjunto de prueba
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  print("RMSE en conjunto de prueba:", rmse)
  r2 = r2_score(y_test, y_pred)
  print("Coeficiente de determinación (R2):", r2)
  best_params = grid_search.best_params_
  best_model = grid_search.best_estimator_
  print(best_params)

  dump(best_model, best_model_dir)
  dump(best_params, best_param_dir)
  results = grid_search.cv_results_
  # Guardar los resultados en un archivo
  with open(results_dir, 'wb') as f:
      pickle.dump(results, f)

  mlp_loaded = joblib.load(best_model_dir)
  predicciones = mlp_loaded.predict(X_test)
  [r2, RMSE, MAE, MAPE] = metrics(y_test, predicciones)
  return [r2, RMSE, MAE, MAPE]



def RFR_grid(X,y, station_name, fecha, best_model_dir, best_param_dir, time_steps, time_future, results_dir):

  X_seq, y_seq = create_sequences2(X, y, time_steps, time_future)
  # Dividir los datos en conjunto de entrenamiento y prueba
  X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
  #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Definir el modelo RandomForestRegressor
  rf = RandomForestRegressor(random_state=42)

  # Definir el diccionario de parámetros para GridSearchCV
  param_grid = {
      'n_estimators': [50, 100, 200],
      'max_depth': [None, 10, 20, 30],
      'min_samples_split': [2, 5, 10],
      'min_samples_leaf': [1, 2, 4],
      'bootstrap': [True, False]
  }

  # Configurar GridSearchCV con n_jobs=-1 para usar todos los núcleos disponibles
  grid_search = GridSearchCV(rf, param_grid, scoring='r2', cv=5, verbose=1, n_jobs=-1, return_train_score=True)

  # Ajustar GridSearchCV a los datos de entrenamiento
  grid_search.fit(X_train, y_train)

  results = grid_search.cv_results_
  # Guardar los resultados en un archivo
  with open(results_dir, 'wb') as f:
      pickle.dump(results, f)
  # Mejor conjunto de parámetros encontrados
  print("Mejores parámetros:", grid_search.best_params_)

  # Obtener el mejor modelo entrenado
  best_model = grid_search.best_estimator_
  best_params = grid_search.best_params_
  # Predecir en el conjunto de prueba
  y_pred = best_model.predict(X_test)
  # Calcular el RMSE en el conjunto de prueba
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  print("RMSE en conjunto de prueba:", rmse)
  r2 = r2_score(y_test, y_pred)
  print("Coeficiente de determinación (R2):", r2)

  dump(best_model, best_model_dir)
  dump(best_params, best_param_dir)

  # Cargar el modelo guardado
  RFR_loaded = joblib.load(best_model_dir)
  # Usar el modelo cargado para hacer predicciones
  predicciones = RFR_loaded.predict(X_test)
  [r2, RMSE, MAE, MAPE] = metrics(y_test, predicciones)
  return [r2, RMSE, MAE, MAPE]


def XGBoost_grid(X,y, station_name, fecha, best_xgb_model_dir, best_xgb_params_dir, time_steps, time_future, results_dir):

  X_seq, y_seq = create_sequences2(X, y, time_steps, time_future)
  # Dividir los datos en conjunto de entrenamiento y prueba
  X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
  #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Definir el modelo XGBoost
  xgb_model = XGBRegressor()

  # Definir los parámetros para GridSearchCV
  param_grid = {
      'n_estimators': [100, 200, 300],
      'max_depth': [3, 4, 5],
      'learning_rate': [0.01, 0.05, 0.1]
  }

  # Inicializar GridSearchCV
  grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='r2',return_train_score=True)

  # Entrenar GridSearchCV
  grid_search.fit(X_train, y_train)

  results = grid_search.cv_results_
  # Guardar los resultados en un archivo
  with open(results_dir, 'wb') as f:
      pickle.dump(results, f)

  # Obtener los mejores parámetros
  best_params = grid_search.best_params_
  print("Mejores parámetros encontrados:", best_params)

  # Entrenar el modelo final con los mejores parámetros
  best_xgb_model = XGBRegressor(**best_params)
  best_xgb_model.fit(X_train, y_train)

  # Evaluar el modelo en el conjunto de prueba
  rmse = np.sqrt(mean_squared_error(y_test, best_xgb_model.predict(X_test)))
  print("RMSE en conjunto de prueba:", rmse)
  # Guardar el modelo
  joblib.dump(best_xgb_model, best_xgb_model_dir)
  joblib.dump(best_params, best_xgb_params_dir)

  loaded_model = joblib.load(best_xgb_model_dir)
  predicciones = loaded_model.predict(X_test)
  r2 = r2_score(y_test, predicciones)
  print("Coeficiente de determinación (R2):", r2)
  [r2, RMSE, MAE, MAPE] = metrics(y_test, predicciones)
  return [r2, RMSE, MAE, MAPE]


def RNN_grid(X,y,station_name,time_steps, fecha, RNN_dir, time_future, results_dir):

  # Define tu propio estimador personalizado
  class CustomKerasRegressor(BaseEstimator, RegressorMixin):
      def __init__(self, units=64, epochs=10, batch_size=32):
          self.units = units
          self.epochs = epochs
          self.batch_size = batch_size
          self.model = None

      def fit(self, X, y):
          input_shape = X.shape[1:]
          self.model = Sequential([
              SimpleRNN(self.units, input_shape=input_shape),
              Dense(1)
          ])
          #'mean_squared_error'
          self.model.compile(optimizer='adam', loss='r2')

          # Definir ModelCheckpoint para guardar los mejores pesos
          checkpoint_filepath = 'model_checkpoint.h5'
          checkpoint_callback = ModelCheckpoint(checkpoint_filepath, save_weights_only=True, save_best_only=True, monitor='val_loss', mode='min', verbose=1, return_train_score=True)

          # Entrenar el modelo
          self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2, callbacks=[checkpoint_callback])

          # Cargar los mejores pesos
          self.model.load_weights(checkpoint_filepath)
          return self

      def predict(self, X):
          return self.model.predict(X)

      def score(self, X, y):
          y_pred = self.predict(X)
          #return -mean_squared_error(y, y_pred)
          return r2_score(y, y_pred)


  X_seq, y_seq = create_sequences(X, y, time_steps, time_future)

  # Dividir los datos en conjunto de entrenamiento y prueba
  X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

  # Definir los parámetros a buscar
  param_grid = {
      'units': [32, 64, 128],
      'epochs': [200, 300,400],
      'batch_size': [32, 64]
  }

  # Convertir la métrica de evaluación (MSE)
  scoring = make_scorer(mean_squared_error, greater_is_better=False)

  # Definir la división de series temporales para cross-validation
  tscv = TimeSeriesSplit(n_splits=5)

  # Crear un modelo Keras personalizado
  custom_keras_regressor = CustomKerasRegressor()

  # Realizar la búsqueda de hiperparámetros con GridSearchCV
  grid_search = GridSearchCV(estimator=custom_keras_regressor, param_grid=param_grid, cv=tscv, scoring=scoring, verbose=1)
  grid_search.fit(X_train, y_train)

  results = grid_search.cv_results_
  # Guardar los resultados en un archivo
  with open(results_dir, 'wb') as f:
      pickle.dump(results, f)

  # Obtener el mejor modelo
  best_model = grid_search.best_estimator_
  # Guardar el mejor modelo y sus pesos
  best_model.model.save(RNN_dir)
  # Cargar el mejor modelo y sus pesos
  loaded_model = load_model(RNN_dir)
  # Evaluación opcional del modelo cargado
  #mse = loaded_model.evaluate(X_test, y_test)
  #print("Mean Squared Error on Test Set:", mse)

  predicciones = loaded_model.predict(X_test)
  r2 = r2_score(y_test, predicciones)
  print("Coeficiente de determinación (R2):", r2)
  [r2, RMSE, MAE, MAPE] = metrics(y_test, predicciones)
  return [r2, RMSE, MAE, MAPE]

## Entrenamos con los datos del tráfico (considerando solo la calle más cercana)

In [ ]:
dir = '/content/drive/MyDrive/docto/A24/Tesis A24/Datos/air_pollutionANDtraffic_data'
dir_metrics_result = '/content/drive/MyDrive/docto/A24/Tesis A24/modelos/metric_results.csv'
files = os.listdir(dir)
files

['air_traffic_MER.csv', 'air_traffic_UIZ.csv']

In [ ]:
dir = '/content/drive/MyDrive/docto/A24/Tesis A24/Datos/air_pollutionANDtraffic_data'
dir_base = '/content/drive/MyDrive/docto/A24/Tesis A24/Modelos/'
dir_metrics_result = dir_base + 'models_sequence/metric_results.csv'
files = os.listdir(dir)
print(files)
metric_results_list= []
column_names = ['MODEL', 'TRAFFIC_DATA','TARGET', 'STATION','R2', 'RMSE', 'MAE', 'MAPE', 'LEN_X', 'DATE','TIME_STEPS', 'TIME_FUTURE','HORA']
for station in files:
  target = "CO"
  X, y, df, file_path = leer_archivo(station, target)
  print(file_path)
  station_name = station[12:-4]
  fecha = '06-06-2024'
  time_steps = 12
  time_future =1
  results_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/results_MLP_'+'timesteps_'+target+'_'+str(time_future)+'_'+'_'+station_name +'_'+fecha+'.pkl'
  best_model_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/best_model_SEQUENCE_MLP_'+str(time_steps)+'timesteps_'+target+'_'+station_name +'_'+fecha+'.joblib'
  best_param_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/best_params_MLP_'+target+'_'+station_name +'_'+fecha+'.joblib'
  [r2, RMSE, MAE, MAPE] = MLP_grid(X,y, station_name, fecha, best_model_dir, best_param_dir, time_steps, time_future, results_dir)
  utc_now = datetime.now(pytz.utc)
  mexico_city_tz = pytz.timezone('America/Mexico_City')
  mexico_city_now = utc_now.astimezone(mexico_city_tz)
  hora_formateada = mexico_city_now.strftime('%Y-%m-%d %H:%M')
  metric_results_list = []
  metric_results_list.append(['MLP', 'NEAREST_STREET', target,station_name, r2, RMSE, MAE, MAPE, len(X), fecha, time_steps, time_future, hora_formateada])
  metric_results_df = pd.DataFrame(metric_results_list, columns=column_names)
  last_metrics_result = pd.read_csv(dir_metrics_result)
  all_metrics = pd.concat([last_metrics_result, metric_results_df])
  all_metrics.to_csv(dir_metrics_result, index=False)
  results_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/results_RFR_'+'timesteps_'+target+'_'+str(time_future)+'_'+'_'+station_name +'_'+fecha+'.pkl'
  best_model_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/best_model_RFR_'+str(time_steps)+'timesteps_'+target+'_'+station_name +'_'+fecha+'.joblib'
  best_param_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/best_params_RFR_'+str(time_steps)+'timesteps_'+target+'_'+station_name +'_'+fecha+'.joblib'
  metric_results_list = []
  [r2, RMSE, MAE, MAPE] = RFR_grid(X,y, station, fecha, best_model_dir, best_param_dir, time_steps, time_future, results_dir)
  utc_now = datetime.now(pytz.utc)
  mexico_city_tz = pytz.timezone('America/Mexico_City')
  mexico_city_now = utc_now.astimezone(mexico_city_tz)
  hora_formateada = mexico_city_now.strftime('%Y-%m-%d %H:%M')
  metric_results_list.append(['RFR', 'NEAREST_STREET', target,station_name, r2, RMSE, MAE, MAPE, len(X), fecha, time_steps, time_future, hora_formateada])
  metric_results_df = pd.DataFrame(metric_results_list, columns=column_names)
  last_metrics_result = pd.read_csv(dir_metrics_result)
  all_metrics = pd.concat([last_metrics_result, metric_results_df])
  all_metrics.to_csv(dir_metrics_result, index=False)
  results_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/results_XGB_'+'timesteps_'+target+'_'+str(time_future)+'_'+'_'+station_name +'_'+fecha+'.pkl'
  best_xgb_model_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/best_model_XGBoost_'+str(time_steps)+'timesteps_'+target+'_'+station_name +'_'+fecha+'.pkl'
  best_xgb_params_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/best_paramas_XGBoost_'+str(time_steps)+'timesteps_'+target+'_'+station_name +'_'+fecha+'.pkl'
  [r2, RMSE, MAE, MAPE] = XGBoost_grid(X,y, station, fecha, best_xgb_model_dir, best_xgb_params_dir, time_steps, time_future, results_dir)
  utc_now = datetime.now(pytz.utc)
  mexico_city_tz = pytz.timezone('America/Mexico_City')
  mexico_city_now = utc_now.astimezone(mexico_city_tz)
  hora_formateada = mexico_city_now.strftime('%Y-%m-%d %H:%M')
  metric_results_list = []
  metric_results_list.append(['XGBoost', 'NEAREST_STREET', target,station_name, r2, RMSE, MAE, MAPE, len(X), fecha, time_steps, time_future, hora_formateada])
  metric_results_df = pd.DataFrame(metric_results_list, columns=column_names)
  last_metrics_result = pd.read_csv(dir_metrics_result)
  all_metrics = pd.concat([last_metrics_result, metric_results_df])
  all_metrics.to_csv(dir_metrics_result, index=False)
  results_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/results_RNN_'+'timesteps_'+target+'_'+str(time_future)+'_'+'_'+station_name +'_'+fecha+'.pkl'
  RNN_dir = dir_base+'models_sequence/MODELS_air_pollution_AND_traffic_data/best_model_RNN_'+str(time_steps)+'timesteps_'+target+'_'+station_name+'_'+fecha+'.h5'
  #[r2, RMSE, MAE, MAPE] = RNN_grid(X,y, station, time_steps, fecha, RNN_dir, time_future, results_dir)
  #utc_now = datetime.now(pytz.utc)
  #mexico_city_tz = pytz.timezone('America/Mexico_City')
  #mexico_city_now = utc_now.astimezone(mexico_city_tz)
  #hora_formateada = mexico_city_now.strftime('%Y-%m-%d %H:%M')
  #metric_results_list = []
  #metric_results_list.append(['RNN', 'NEAREST_STREET', target,station_name, r2, RMSE, MAE, MAPE, len(X), fecha, time_future, hora_formateada])
  #metric_results_df = pd.DataFrame(metric_results_list, columns=column_names)
  #last_metrics_result = pd.read_csv(dir_metrics_result)
  #all_metrics = pd.concat([last_metrics_result, metric_results_df])
  #all_metrics.to_csv(dir_metrics_result, index=False)
  print(all_metrics)

## Ahora entrenamos los mismos modelos pero sin los datos del tráfico

In [7]:
dir = '/content/drive/MyDrive/docto/A24/Tesis A24/Datos/air_pollutionANDtraffic_data'
dir_base = '/content/drive/MyDrive/docto/A24/Tesis A24/Modelos/'
dir_metrics_result = dir_base + 'models_sequence/metric_results.csv'
files = os.listdir(dir)
print(files)
metric_results_list= []
column_names = ['MODEL', 'TRAFFIC_DATA','TARGET', 'STATION','R2', 'RMSE', 'MAE', 'MAPE', 'LEN_X', 'DATE','TIME_STEPS', 'TIME_FUTURE','HORA']
for station in files:
  target = "CO"
  X, y, df, file_path = leer_archivo(station, target)
  X = X.drop(columns=['traffic'])
  print(file_path)
  station_name = station[12:-4]
  fecha = '06-06-2024'
  time_steps = 12
  time_future = 1
  """
  results_dir = dir_base+'models_sequence/MODELS_air_pollution/results_MLP_'+'timesteps_'+target+'_'+str(time_future)+'_'+'_'+station_name +'_'+fecha+'.pkl'
  best_model_dir = dir_base+'models_sequence/MODELS_air_pollution/best_model_SEQUENCE_MLP_'+str(time_steps)+'timesteps_'+target+'_'+station_name +'_'+fecha+'.joblib'
  best_param_dir = dir_base+'models_sequence/MODELS_air_pollution/best_params_MLP_'+target+'_'+station_name +'_'+fecha+'.joblib'
  [r2, RMSE, MAE, MAPE] = MLP_grid(X,y, station_name, fecha, best_model_dir, best_param_dir, time_steps, time_future, results_dir)
  utc_now = datetime.now(pytz.utc)
  mexico_city_tz = pytz.timezone('America/Mexico_City')
  mexico_city_now = utc_now.astimezone(mexico_city_tz)
  hora_formateada = mexico_city_now.strftime('%Y-%m-%d %H:%M')
  metric_results_list = []
  metric_results_list.append(['MLP', 'NO', target,station_name, r2, RMSE, MAE, MAPE, len(X), fecha, time_steps, time_future, hora_formateada])
  metric_results_df = pd.DataFrame(metric_results_list, columns=column_names)
  last_metrics_result = pd.read_csv(dir_metrics_result)
  all_metrics = pd.concat([last_metrics_result, metric_results_df])
  all_metrics.to_csv(dir_metrics_result, index=False)
  """
  results_dir = dir_base+'models_sequence/MODELS_air_pollution/results_RFR_'+'timesteps_'+target+'_'+str(time_future)+'_'+'_'+station_name +'_'+fecha+'.pkl'
  best_model_dir = dir_base+'models_sequence/MODELS_air_pollution/best_model_RFR_'+str(time_steps)+'timesteps_'+target+'_'+station_name +'_'+fecha+'.joblib'
  best_param_dir = dir_base+'models_sequence/MODELS_air_pollution/best_params_RFR_'+str(time_steps)+'timesteps_'+target+'_'+station_name +'_'+fecha+'.joblib'

  metric_results_list = []
  [r2, RMSE, MAE, MAPE] = RFR_grid(X,y, station, fecha, best_model_dir, best_param_dir, time_steps, time_future, results_dir)
  utc_now = datetime.now(pytz.utc)
  mexico_city_tz = pytz.timezone('America/Mexico_City')
  mexico_city_now = utc_now.astimezone(mexico_city_tz)
  hora_formateada = mexico_city_now.strftime('%Y-%m-%d %H:%M')
  metric_results_list.append(['RFR', 'NO', target,station_name, r2, RMSE, MAE, MAPE, len(X), fecha, time_steps, time_future, hora_formateada])
  metric_results_df = pd.DataFrame(metric_results_list, columns=column_names)
  last_metrics_result = pd.read_csv(dir_metrics_result)
  all_metrics = pd.concat([last_metrics_result, metric_results_df])
  all_metrics.to_csv(dir_metrics_result, index=False)

  results_dir = dir_base+'models_sequence/MODELS_air_pollution/results_XGB_'+'timesteps_'+target+'_'+str(time_future)+'_'+'_'+station_name +'_'+fecha+'.pkl'
  best_xgb_model_dir = dir_base+'models_sequence/MODELS_air_pollution/best_model_XGBoost_'+str(time_steps)+'timesteps_'+target+'_'+station_name +'_'+fecha+'.pkl'
  best_xgb_params_dir = dir_base+'models_sequence/MODELS_air_pollution/best_paramas_XGBoost_'+str(time_steps)+'timesteps_'+target+'_'+station_name +'_'+fecha+'.pkl'
  [r2, RMSE, MAE, MAPE] = XGBoost_grid(X,y, station, fecha, best_xgb_model_dir, best_xgb_params_dir, time_steps, time_future, results_dir)
  utc_now = datetime.now(pytz.utc)
  mexico_city_tz = pytz.timezone('America/Mexico_City')
  mexico_city_now = utc_now.astimezone(mexico_city_tz)
  hora_formateada = mexico_city_now.strftime('%Y-%m-%d %H:%M')
  metric_results_list = []
  metric_results_list.append(['XGBoost', 'NO', target,station_name, r2, RMSE, MAE, MAPE, len(X), fecha, time_steps, time_future, hora_formateada])
  metric_results_df = pd.DataFrame(metric_results_list, columns=column_names)
  last_metrics_result = pd.read_csv(dir_metrics_result)
  all_metrics = pd.concat([last_metrics_result, metric_results_df])
  all_metrics.to_csv(dir_metrics_result, index=False)
  results_dir = dir_base+'models_sequence/MODELS_air_pollution/results_RNN_'+'timesteps_'+target+'_'+str(time_future)+'_'+'_'+station_name +'_'+fecha+'.pkl'
  RNN_dir = dir_base+'models_sequence/MODELS_air_pollution/best_model_RNN_'+str(time_steps)+'timesteps_'+target+'_'+station_name+'_'+fecha+'.h5'
  #[r2, RMSE, MAE, MAPE] = RNN_grid(X,y, station, time_steps, fecha, RNN_dir, time_future, results_dir)
  #utc_now = datetime.now(pytz.utc)
  #mexico_city_tz = pytz.timezone('America/Mexico_City')
  #mexico_city_now = utc_now.astimezone(mexico_city_tz)
  #hora_formateada = mexico_city_now.strftime('%Y-%m-%d %H:%M')
  #metric_results_list = []
  #metric_results_list.append(['RNN', 'NO', target,station_name, r2, RMSE, MAE, MAPE, len(X), fecha, time_future, hora_formateada])
  #metric_results_df = pd.DataFrame(metric_results_list, columns=column_names)
  #last_metrics_result = pd.read_csv(dir_metrics_result)
  #all_metrics = pd.concat([last_metrics_result, metric_results_df])
  #all_metrics.to_csv(dir_metrics_result, index=False)

  print(all_metrics)

['air_traffic_MER.csv', 'air_traffic_UIZ.csv']
/content/drive/MyDrive/docto/A24/Tesis A24/Datos/air_pollutionANDtraffic_data/air_traffic_MER.csv
Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


KeyboardInterrupt: 

In [ ]:
import pickle
dir = '/content/drive/MyDrive/docto/A24/Tesis A24/Modelos/models_sequence/MODELS_air_pollution_AND_traffic_data/results_XGB_CO_UIZ_06-06-2024.pkl'
# Cargar los resultados desde el archivo
with open(dir, 'rb') as f:
    loaded_results = pickle.load(f)

# Mostrar los resultados de los puntajes de entrenamiento
train_scores = loaded_results['mean_train_score']
print("Mean train scores for each parameter combination:")
for mean_train_score in train_scores:
    print(mean_train_score)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.05, 0.1]
}

# Cargar un conjunto de datos de ejemplo
data = load_iris()
X = data.data
y = data.target

# Definir un modelo
model = RandomForestClassifier()

# Definir una rejilla de parámetros
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [1, 10, 20]
}

# Crear el objeto GridSearchCV con return_train_score=True
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, return_train_score=True)

# Ajustar el modelo a los datos
grid_search.fit(X, y)

# Obtener los resultados
results = grid_search.cv_results_

#results = loaded_results
# Extraer los puntajes de entrenamiento y validación
train_scores = np.array(results['mean_train_score']).reshape(len(param_grid['max_depth']), len(param_grid['n_estimators']))
test_scores = np.array(results['mean_test_score']).reshape(len(param_grid['max_depth']), len(param_grid['n_estimators']))

# Graficar los resultados
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# Crear rejilla de parámetros
n_estimators, max_depth = np.meshgrid(param_grid['n_estimators'], param_grid['max_depth'])

# Graficar superficies
ax.plot_surface(n_estimators, max_depth, train_scores, cmap='viridis', label='Train Score')
ax.plot_surface(n_estimators, max_depth, test_scores, cmap='plasma', label='Test Score')

# Etiquetas y título
ax.set_xlabel('Number of Estimators')
ax.set_ylabel('Max Depth')
ax.set_zlabel('Mean Score')
ax.set_title('Grid Search Results')

# Añadir leyenda
ax.legend()

# Mostrar la gráfica
plt.show()


In [ ]:
results